# ETL Code

This code uses an API to get data from the WorldBank dataset and then saves it into tables in a database for use in the main code. 

In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio
import requests

In [2]:
# Requesting the indicators for the topic Political Stability. The 3 is the key for the Political Stability Data.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=3")
indicatorsJSON = indicators.json()

In [3]:
# Get all Worldwide Governance indicators so we know what is available
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=3&per_page=500")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

('CC.EST', 'Control of Corruption: Estimate')
('CC.NO.SRC', 'Control of Corruption: Number of Sources')
('CC.PER.RNK', 'Control of Corruption: Percentile Rank')
('CC.PER.RNK.LOWER', 'Control of Corruption: Percentile Rank, Lower Bound of 90% Confidence Interval')
('CC.PER.RNK.UPPER', 'Control of Corruption: Percentile Rank, Upper Bound of 90% Confidence Interval')
('CC.STD.ERR', 'Control of Corruption: Standard Error')
('GE.EST', 'Government Effectiveness: Estimate')
('GE.NO.SRC', 'Government Effectiveness: Number of Sources')
('GE.PER.RNK', 'Government Effectiveness: Percentile Rank')
('GE.PER.RNK.LOWER', 'Government Effectiveness: Percentile Rank, Lower Bound of 90% Confidence Interval')
('GE.PER.RNK.UPPER', 'Government Effectiveness: Percentile Rank, Upper Bound of 90% Confidence Interval')
('GE.STD.ERR', 'Government Effectiveness: Standard Error')
('PV.EST', 'Political Stability and Absence of Violence/Terrorism: Estimate')
('PV.NO.SRC', 'Political Stability and Absence of Violence

In [4]:
# Use the indicator code to define the "indicator" variable.  We can change this to get information on each of the indicators.
indicator = "PV.STD.ERR"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Political Stability and Absence of Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence, including terrorism.  Standard error indicates the precision of the estimate of governance.  Larger values of the standard error indicate less precise estimates.  A 90 percent confidence interval for the governance estimate is given by the estimate +/- 1.64 times the standard error.


In [5]:
#Get a list of all the location abbreviations so we know what they are for limiting selections.
dlocations = requests.get("http://api.worldbank.org/v2/sources/3/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(dlocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "country"])     
for i in range(0,listLen):
    code = dlocations[i]["id"]
    name = dlocations[i]["value"]
    df = df.append({"id":code, "country":name}, ignore_index = True)
dlocationsList = df

# See first few items in the dataframe
dlocationsList.head()

,id,country
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania


In [6]:
#Select the indicator, location, and time you would like for the API call
# Selecting the indicator
indicatorSelection = {'PV.EST':'Political Stability and Absence of Violence/Terrorism: Estimate',
                     'PV.STD.ERR': 'Political Stability and Absence of Violence/Terrorism: Standard Error'}
# Select the countries or regions
locationSelection = ["all"]
# Selecting the time frame
timeSelection = (datetime.datetime(2010, 1, 1), datetime.datetime(2020, 12, 31))
#Set the source id number for the Worldwide Governance Data
source_id=3

In [7]:
# Make the API call and assign the resulting DataFrame to "PT_data"
#Notes from wbdata: 
"""
    Convenience function to download a set of indicators and  merge them into a
        pandas DataFrame.  The index will be the same as if calls were made to
        get_data separately.
    :indicators: A dictionary where the keys are desired indicators and the
        values are the desired column names
    :country: a country code, sequence of country codes, or "all" (default)
    :data_date: the desired date as a datetime object or a 2-sequence with
        start and end dates
    :freq: the desired periodicity of the data, one of 'Y' (yearly), 'M'
        (monthly), or 'Q' (quarterly). The indicator may or may not support the
        specified frequency.
    :source: the specific source to retrieve data from (defaults on API to 2,
        World Development Indicators)
    :convert_date: if True, convert date field to a datetime.datetime object.
    :keep_levels: if True don't reduce the number of index levels returned if
        only getting one date or country
    :cache: use the cache
    :returns: a WBDataFrame
    """
PT_data = wbdata.get_dataframe(indicatorSelection,
                            source=source_id,
                            country = locationSelection, 
                            data_date = timeSelection, 
                            convert_date = False)

In [8]:
PT_data.head()

Political Stability and Absence of Violence/Terrorism: Estimate  \
country     date                                                                    
Afghanistan 2019                                          -2.649407                 
            2018                                          -2.751611                 
            2017                                          -2.800609                 
            2016                                          -2.671054                 
            2015                                          -2.571222                 

                  Political Stability and Absence of Violence/Terrorism: Standard Error  
country     date                                                                         
Afghanistan 2019                                           0.241833                      
            2018                                           0.226781                      
            2017                                           0.227994                      
            2016                                           0.219708                      
            2015                                           0.215604

In [9]:
# Turn multiindex into columns
PT_data.reset_index(inplace=True)  
PT_data.head()

,country,date,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error
0,Afghanistan,2019,-2.649407,0.241833
1,Afghanistan,2018,-2.751611,0.226781
2,Afghanistan,2017,-2.800609,0.227994
3,Afghanistan,2016,-2.671054,0.219708
4,Afghanistan,2015,-2.571222,0.215604


In [10]:
#Merge Political Stability data with dlocationsList to get the ISO code for each country
PT_data_with_loc = pd.merge(dlocationsList, PT_data, on='country')
PT_data_with_loc.head(-1)

,id,country,date,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error
0,ABW,Aruba,2019,1.357372,0.271196
1,ABW,Aruba,2018,1.362981,0.251480
2,ABW,Aruba,2017,1.334757,0.254837
3,ABW,Aruba,2016,1.284199,0.244329
4,ABW,Aruba,2015,1.230921,0.241163
...,...,...,...,...,...
2134,ZWE,Zimbabwe,2015,-0.618156,0.202983
2135,ZWE,Zimbabwe,2014,-0.713570,0.196386
2136,ZWE,Zimbabwe,2013,-0.665950,0.228020
2137,ZWE,Zimbabwe,2012,-0.778662,0.221984


In [11]:
#make fucntion to get lat and lon of each country for plotting from country code
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="http")

def get_lat(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return loc.latitude
    except:
        # Return missing value
        return np.nan
    
def get_lon(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return loc.longitude
    except:
        # Return missing value
        return np.nan

In [12]:
# Remove countries with no Political tension data
PT_data_with_loc.dropna(axis=0, inplace=True)
PT_data_with_loc.head()

,id,country,date,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error
0,ABW,Aruba,2019,1.357372,0.271196
1,ABW,Aruba,2018,1.362981,0.251480
2,ABW,Aruba,2017,1.334757,0.254837
3,ABW,Aruba,2016,1.284199,0.244329
4,ABW,Aruba,2015,1.230921,0.241163


In [13]:
# Make a new dataframe storing the country name, country code, lat and lon for each country to use with other datasets
country_data = PT_data_with_loc.drop(['Political Stability and Absence of Violence/Terrorism: Estimate', 'Political Stability and Absence of Violence/Terrorism: Standard Error'], axis=1)
country_df = country_data.drop('date', axis=1)
country_df.drop_duplicates(['country'], inplace=True)

In [14]:
#Get Lat lon of each country based on id using function above.

country_df['lat'] = country_df.apply(lambda row : get_lat(row['country']), axis=1)
country_df['lon'] = country_df.apply(lambda row : get_lon(row['country']), axis=1)

country_df.head()

,id,country,lat,lon
0,ABW,Aruba,12.490300,-69.960984
10,AFG,Afghanistan,33.768006,66.238514
20,AGO,Angola,-11.877577,17.569124
30,AIA,Anguilla,18.195495,-63.075023
40,ALB,Albania,41.000028,19.999962


In [15]:
#Remove any countries with nan values for lat or lon
country_df.dropna(axis=0, inplace=True)
country_df.head()

,id,country,lat,lon
0,ABW,Aruba,12.490300,-69.960984
10,AFG,Afghanistan,33.768006,66.238514
20,AGO,Angola,-11.877577,17.569124
30,AIA,Anguilla,18.195495,-63.075023
40,ALB,Albania,41.000028,19.999962


In [16]:
#merge country_df and co2_data now that we have lat lon for each country
PT_new_df = pd.merge(PT_data_with_loc, country_df, on='country')
PT_new_df.head()

,id_x,country,date,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error,id_y,lat,lon
0,ABW,Aruba,2019,1.357372,0.271196,ABW,12.4903,-69.960984
1,ABW,Aruba,2018,1.362981,0.251480,ABW,12.4903,-69.960984
2,ABW,Aruba,2017,1.334757,0.254837,ABW,12.4903,-69.960984
3,ABW,Aruba,2016,1.284199,0.244329,ABW,12.4903,-69.960984
4,ABW,Aruba,2015,1.230921,0.241163,ABW,12.4903,-69.960984


In [17]:
#remove extra id column
PT_new_df.drop('id_y', axis=1, inplace=True)
PT_new_df.head()

,id_x,country,date,Political Stability and Absence of Violence/Terrorism: Estimate,Political Stability and Absence of Violence/Terrorism: Standard Error,lat,lon
0,ABW,Aruba,2019,1.357372,0.271196,12.4903,-69.960984
1,ABW,Aruba,2018,1.362981,0.251480,12.4903,-69.960984
2,ABW,Aruba,2017,1.334757,0.254837,12.4903,-69.960984
3,ABW,Aruba,2016,1.284199,0.244329,12.4903,-69.960984
4,ABW,Aruba,2015,1.230921,0.241163,12.4903,-69.960984


In [18]:
#Change df column names for easier processing
PT_new_df.rename(columns={"Political Stability and Absence of Violence/Terrorism: Estimate": "Political_Stability", "Political Stability and Absence of Violence/Terrorism: Standard Error": "Political_Stability_SE", "id_x": "id"}, inplace=True)
PT_new_df.head()

,id,country,date,Political_Stability,Political_Stability_SE,lat,lon
0,ABW,Aruba,2019,1.357372,0.271196,12.4903,-69.960984
1,ABW,Aruba,2018,1.362981,0.251480,12.4903,-69.960984
2,ABW,Aruba,2017,1.334757,0.254837,12.4903,-69.960984
3,ABW,Aruba,2016,1.284199,0.244329,12.4903,-69.960984
4,ABW,Aruba,2015,1.230921,0.241163,12.4903,-69.960984


In [43]:
print(len(PT_new_df))

2034


# Get the Predictor Variables from the Sustainability Goals World Bank Dataset

In [19]:
# Requesting the indicators for the topic Political Stability.  The 46 is the key for the Political Stability Data.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=46")
indicatorsJSON = indicators.json()

In [20]:
# Get all Sustainable Development Variables so we know what is available
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=46&per_page=500")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command

('AG.LND.FRST.K2', 'Forest area (sq. km)')
('AG.LND.FRST.ZS', 'Forest area (% of land area)')
('AG.YLD.CREL.KG', 'Cereal yield (kg per hectare)')
('BN.CAB.XOKA.GD.ZS', 'Current account balance (% of GDP)')
('BN.KLT.PTXL.CD', 'Portfolio Investment, net (BoP, current US$)')
('BX.KLT.DINV.CD.WD', 'Foreign direct investment, net inflows (BoP, current US$)')
('BX.KLT.DINV.WD.GD.ZS', 'Foreign direct investment, net inflows (% of GDP)')
('BX.TRF.PWKR.DT.GD.ZS', 'Personal remittances, received (% of GDP)')
('DT.DOD.DECT.GN.ZS', 'External debt stocks (% of GNI)')
('DT.ODA.ALLD.CD', 'Net official development assistance and official aid received (current US$)')
('DT.ODA.ODAT.CD', 'Net official development assistance received (current US$)')
('DT.ODA.ODAT.CD1', 'Net official development assistance received (current US$)')
('DT.ODA.ODAT.KD', 'Net official development assistance received (constant 2018 US$)')
('DT.TDS.DPPF.XP.ZS', 'Debt service (PPG and IMF only, % of exports of goods, services and 

In [21]:
# Use the indicator code to define the "indicator" variable.  We can change this to get information on each of the indicators.
indicator = "SH.SVR.WAST.FE.ZS"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Prevalence of severe wasting, female, is the proportion of girls under age 5 whose weight for height is more than three standard deviations below the median for the international reference population ages 0-59.


In [22]:
#Get a list of all the location abbreviations from the sustainable development dataset so we know what they are for limiting selections.
dlocations = requests.get("http://api.worldbank.org/v2/sources/46/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(dlocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "country"])     
for i in range(0,listLen):
    code = dlocations[i]["id"]
    name = dlocations[i]["value"]
    df = df.append({"id":code, "country":name}, ignore_index = True)
dlocationsList = df

# See first few items in the dataframe
dlocationsList.head()

,id,country
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,ALB,Albania
4,AND,Andorra


In [26]:
#Select the indicator, location, and time you would like for the API call
# Selecting the indicator
indicatorSelection = {'AG.LND.FRST.ZS':'Forest area (% of land area)', 
                      'AG.YLD.CREL.KG':'Cereal yield (kg per hectare)',
                      'BN.CAB.XOKA.GD.ZS':'Current account balance (% of GDP)', 
                      'BX.KLT.DINV.WD.GD.ZS':'Foreign direct investment, net inflows (% of GDP)',
                      'DT.ODA.ALLD.CD':'Net official development assistance and official aid received (current US$)',
                      'EG.CFT.ACCS.ZS':'Access to clean fuels and technologies for cooking (% of population)',
                      'EG.ELC.ACCS.ZS':'Access to electricity (% of population)',
                      'EG.ELC.RNEW.ZS':'Renewable electricity output (% of total electricity output)',
                      'EN.ATM.CO2E.PC':'CO2 emissions (metric tons per capita)',
                      'EN.ATM.PM25.MC.ZS':'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)',
                      'EN.MAM.THRD.NO':'Mammal species, threatened',
                      'EN.POP.SLUM.UR.ZS':'Population living in slums (% of urban population)',
                      'ER.H2O.FWST.ZS':'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources',
                      'ER.H2O.INTR.PC':'Renewable internal freshwater resources per capita (cubic meters)',
                      'ER.LND.PTLD.ZS':'Terrestrial protected areas (% of total land area)',
                      'FB.CBK.BRCH.P5':'Commercial bank branches (per 100,000 adults)',
                      'FM.LBL.BMNY.ZG':'Broad money growth (annual %)',
                      'FP.CPI.TOTL.ZG':'Inflation, consumer prices (annual %)',
                      'FX.OWN.TOTL.ZS':'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
                      'FX.OWN.TOTL.FE.ZS':'Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)',
                      'GB.XPD.RSDV.GD.ZS':'Research and development expenditure (% of GDP)',
                      #'GC.TAX.TOTL.GD.ZS':'Tax revenue (% of GDP)',
                      'IC.BUS.NDNS.ZS':'New business density (new registrations per 1,000 people ages 15-64)',
                      'IE.PPI.ENGY.CD':'Investment in energy with private participation (current US$)',
                      'IE.PPI.WATR.CD':'Investment in water and sanitation with private participation (current US$)',
                      'IT.NET.USER.ZS':'Individuals using the Internet (% of population)',
                      'NE.EXP.GNFS.KD.ZG':'Exports of goods and services (annual % growth)',
                      'NE.EXP.GNFS.ZS':'Exports of goods and services (% of GDP)',
                      'NE.IMP.GNFS.KD.ZG':'Imports of goods and services (annual % growth)',
                      'NY.GDP.MKTP.CD':'GDP (current US$)',
                      'NY.GDP.MKTP.KD.ZG':'GDP growth (annual %)',
                      'NY.GDP.PCAP.CD':'GDP per capita (current US$)',
                      'NY.GDP.PCAP.KD.ZG':'GDP per capita growth (annual %)',
                      'SE.ADT.1524.LT.FE.ZS':'Literacy rate, youth female (% of females ages 15-24)',
                      'SE.ADT.LITR.ZS':'Literacy rate, adult total (% of people ages 15 and above)',
                      'SE.PRE.ENRR':'School enrollment, preprimary (% gross)',
                      'SE.ENR.PRSC.FM.ZS':'School enrollment, primary and secondary (gross), gender parity index (GPI)',
                      'SE.SEC.CMPT.LO.ZS':'Lower secondary completion rate, total (% of relevant age group)',
                      'SE.SEC.CUAT.PO.FE.ZS':'Educational attainment, at least completed post-secondary, population 25+, female (%) (cumulative)',
                      'SE.TER.CUAT.BA.ZS':"Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)",
                      'SE.SEC.UNER.LO.ZS':'Adolescents out of school (% of lower secondary school age)',
                      'SH.ALC.PCAP.LI':'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                      'SH.ANM.ALLW.ZS':'Prevalence of anemia among women of reproductive age (% of women ages 15-49)',
                      'SH.DYN.AIDS.ZS':'Prevalence of HIV, total (% of population ages 15-49)',
                      'SH.DYN.MORT':'Mortality rate, under-5 (per 1,000 live births)',
                      'SH.DYN.NCOM.ZS':'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)',
                      'SH.DYN.NMRT':'Mortality rate, neonatal (per 1,000 live births)',
                      'SH.H2O.BASW.ZS':'People using at least basic drinking water services (% of population)',
                      'SH.H2O.SMDW.ZS':'People using safely managed drinking water services (% of population)',
                      'SH.STA.FGMS.ZS':'Female genital mutilation prevalence (%)',
                      'SH.STA.MALN.ZS':'Prevalence of underweight, weight for age (% of children under 5)',
                      'SH.STA.SUIC.P5':'Suicide mortality rate (per 100,000 population)',
                      'SH.UHC.OOPC.10.ZS':'Proportion of population spending more than 10% of household consumption or income on out-of-pocket health care expenditure (%)',
                      'SL.UEM.TOTL.ZS':'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                      'SN.ITK.MSFI.ZS':'Prevalence of moderate or severe food insecurity in the population (%)',
                      'SP.M18.2024.FE.ZS':'Women who were first married by age 18 (% of women ages 20-24)',
                      'SP.URB.GROW':'Urban population growth (annual %)',
                      'VC.BTL.DETH':'Battle-related deaths (number of people)',
                      'VC.IHR.PSRC.P5':'Intentional homicides (per 100,000 people)'
                       }
# Select the countries or regions
locationSelection = ["all"]
# Selecting the time frame
timeSelection = (datetime.datetime(2010, 1, 1), datetime.datetime(2020, 12, 31))
#Set the source id number for the Worldwide Governance Data
source_id=46

In [27]:
# Make the API call and assign the resulting DataFrame to "DF_data"
#Notes from wbdata: 
"""
    Convenience function to download a set of indicators and  merge them into a
        pandas DataFrame.  The index will be the same as if calls were made to
        get_data separately.
    :indicators: An dictionary where the keys are desired indicators and the
        values are the desired column names
    :country: a country code, sequence of country codes, or "all" (default)
    :data_date: the desired date as a datetime object or a 2-sequence with
        start and end dates
    :freq: the desired periodicity of the data, one of 'Y' (yearly), 'M'
        (monthly), or 'Q' (quarterly). The indicator may or may not support the
        specified frequency.
    :source: the specific source to retrieve data from (defaults on API to 2,
        World Development Indicators)
    :convert_date: if True, convert date field to a datetime.datetime object.
    :keep_levels: if True don't reduce the number of index levels returned if
        only getting one date or country
    :cache: use the cache
    :returns: a WBDataFrame
    """
DF_data = wbdata.get_dataframe(indicatorSelection,
                            source=source_id,
                            country = locationSelection, 
                            data_date = timeSelection, 
                            convert_date = False)

In [28]:
DF_data.head()

Forest area (% of land area)  Cereal yield (kg per hectare)  \
country    date                                                                
Arab World 2019                           NaN                            NaN   
           2018                      2.849036                    2011.461649   
           2017                      2.868595                    1869.677439   
           2016                      2.887384                    1705.274933   
           2015                      2.904999                    2057.438839   

                 Current account balance (% of GDP)  \
country    date                                       
Arab World 2019                                 NaN   
           2018                                 NaN   
           2017                                 NaN   
           2016                                 NaN   
           2015                                 NaN   

                 Foreign direct investment, net inflows (% of GDP)  \
country    date                                                      
Arab World 2019                                           1.278438   
           2018                                           1.258055   
           2017                                           1.230380   
           2016                                           1.360062   
           2015                                           1.043033   

                 Net official development assistance and official aid received (current US$)  \
country    date                                                                                
Arab World 2019                                                NaN                             
           2018                                                NaN                             
           2017                                                NaN                             
           2016                                                NaN                             
           2015                                                NaN                             

                 Access to clean fuels and technologies for cooking (% of population)  \
country    date                                                                         
Arab World 2019                                                NaN                      
           2018                                                NaN                      
           2017                                                NaN                      
           2016                                          84.510733                      
           2015                                          84.171596                      

                 Access to electricity (% of population)  \
country    date                                            
Arab World 2019                                89.512282   
           2018                                88.306324   
           2017                                89.623427   
           2016                                88.504261   
           2015                                87.935479   

                 Renewable electricity output (% of total electricity output)  \
country    date                                                                 
Arab World 2019                                                NaN              
           2018                                                NaN              
           2017                                                NaN              
           2016                                                NaN              
           2015                                           2.920702              

                 CO2 emissions (metric tons per capita)  \
country    date                                           
Arab World 2019                                     NaN   
           2018                                4.438716   
           2017                                4.528351  

In [29]:
# Turn multiindex into columns
DF_data.reset_index(inplace=True)  
DF_data.head()

,country,date,Forest area (% of land area),Cereal yield (kg per hectare),Current account balance (% of GDP),"Foreign direct investment, net inflows (% of GDP)",Net official development assistance and official aid received (current US$),Access to clean fuels and technologies for cooking (% of population),Access to electricity (% of population),Renewable electricity output (% of total electricity output),...,Female genital mutilation prevalence (%),"Prevalence of underweight, weight for age (% of children under 5)","Suicide mortality rate (per 100,000 population)",Proportion of population spending more than 10% of household consumption or income on out-of-pocket health care expenditure (%),"Unemployment, total (% of total labor force) (modeled ILO estimate)",Prevalence of moderate or severe food insecurity in the population (%),Women who were first married by age 18 (% of women ages 20-24),Urban population growth (annual %),Battle-related deaths (number of people),"Intentional homicides (per 100,000 people)"
0,Arab World,2019,NaN,NaN,NaN,1.278438,NaN,NaN,89.512282,NaN,...,NaN,NaN,4.180325,NaN,10.207223,NaN,NaN,2.385471,NaN,NaN
1,Arab World,2018,2.849036,2011.461649,NaN,1.258055,NaN,NaN,88.306324,NaN,...,NaN,NaN,4.212684,NaN,10.256579,NaN,NaN,2.414030,20934.0,NaN
2,Arab World,2017,2.868595,1869.677439,NaN,1.230380,NaN,NaN,89.623427,NaN,...,NaN,NaN,4.179462,NaN,10.718294,NaN,NaN,2.473679,37279.0,NaN
3,Arab World,2016,2.887384,1705.274933,NaN,1.360062,NaN,84.510733,88.504261,NaN,...,NaN,NaN,4.258030,NaN,10.507661,NaN,NaN,2.576060,58959.0,NaN
4,Arab World,2015,2.904999,2057.438839,NaN,1.043033,NaN,84.171596,87.935479,2.920702,...,NaN,NaN,4.333516,NaN,10.713793,NaN,NaN,2.695668,68181.0,3.8


# Store the dataframes in a database as tables to access anytime the data is needed

In [31]:
#Get the features to save to the database
sus_features_df = DF_data.copy()

#save the column names to a list
sus_columns=sus_features_df.columns
#rename the columns to make them easier for processing
new_col_names = ['country', 'date', 'Forest_area', 'Cereal_yield', 'Current_account_balance', 'Foreign_direct_investment', 'Net_assistance_and_aid', 'Access_clean_fuels', 'Access_electricity', 'Renewable_electricity_output', 'CO2_emissions', 'air_pollution', 'Mammal_species_threatened', 'Population_in_slums', 'water_stress', 'Renewable_freshwater_resources', 'Terrestrial_protected_areas', 'Commercial_bank_branches', 'Broad_money_growth', 'Inflation', 'Account_ownership_financial_institution', 'Account_ownership_financial_institution_female', 'Research_development_expenditure', 'New_business_density', 'Investment_in_energy', 'Investment_in_clean_water', 'Individuals_using_Internet', 'Exports_of_goods_annual_growth', 'Exports_of_goods', 'Imports_of_goods_annual_growth', 'GDP', 'GDP_growth', 'GDP_per_capita', 'GDP_per_capita_growth', 'Literacy_rate_youth_female', 'Literacy_rate_adult_total', 'School_enrollment_preprimary', 'School_enrollment_primary_and_secondary_GPI', 'Lower_secondary_completion_rate', 'Educational_attainment_post_secondary_female', 'Educational_attainment_Bachelor', 'Adolescents_out_of_school', 'Total_alcohol_consumption_per_capita', 'Prevalence_anemia_among_women', 'Prevalence_HIV', 'Mortality_rate_under_5', 'Mortality_CVD_cancer_diabetes', 'Mortality_rate_neonatal', 'People_using_basic_drinking_water', 'People_using_safely_managed_drinking_water', 'Female_genital_mutilation_prevalence', 'Prevalence_underweight_children_under_five', 'Suicide_mortality_rate', 'High_health_care_expenditure', 'Unemployment', 'Moderate_severe_food_insecurity', 'Women_married_by_age_18', 'Urban_population_growth', 'Battle_related_deaths', 'Intentional_homicides_per_pop']
sus_features_df.columns = new_col_names
sus_features_df.head()

,country,date,Forest_area,Cereal_yield,Current_account_balance,Foreign_direct_investment,Net_assistance_and_aid,Access_clean_fuels,Access_electricity,Renewable_electricity_output,...,Female_genital_mutilation_prevalence,Prevalence_underweight_children_under_five,Suicide_mortality_rate,High_health_care_expenditure,Unemployment,Moderate_severe_food_insecurity,Women_married_by_age_18,Urban_population_growth,Battle_related_deaths,Intentional_homicides_per_pop
0,Arab World,2019,NaN,NaN,NaN,1.278438,NaN,NaN,89.512282,NaN,...,NaN,NaN,4.180325,NaN,10.207223,NaN,NaN,2.385471,NaN,NaN
1,Arab World,2018,2.849036,2011.461649,NaN,1.258055,NaN,NaN,88.306324,NaN,...,NaN,NaN,4.212684,NaN,10.256579,NaN,NaN,2.414030,20934.0,NaN
2,Arab World,2017,2.868595,1869.677439,NaN,1.230380,NaN,NaN,89.623427,NaN,...,NaN,NaN,4.179462,NaN,10.718294,NaN,NaN,2.473679,37279.0,NaN
3,Arab World,2016,2.887384,1705.274933,NaN,1.360062,NaN,84.510733,88.504261,NaN,...,NaN,NaN,4.258030,NaN,10.507661,NaN,NaN,2.576060,58959.0,NaN
4,Arab World,2015,2.904999,2057.438839,NaN,1.043033,NaN,84.171596,87.935479,2.920702,...,NaN,NaN,4.333516,NaN,10.713793,NaN,NaN,2.695668,68181.0,3.8


In [32]:
import psycopg2

# Connecting to the Postgres Server
conn = psycopg2.connect("user=postgres password=122516")
#make it so that every transaction is run independently and a rollback is done automatically if a transaction fails, otherwise a commit occurs.
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT);

In [33]:
# Get cursor object from the database connection
cursor = conn.cursor()

name_Database = "sustain_data"

# Create the crime data database
sqlCreateDatabase = "create database "+name_Database+";"

# Create a table in PostgreSQL database
#cursor.execute(sqlCreateDatabase);

In [34]:
# Connecting to the database (set a new connection)
conn = psycopg2.connect("dbname=sustain_data user=postgres password=122516")
#make it so that every transaction is run independently and a rollback is done automatically if a transaction failed, otherwise a commit occurs.
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()

In [44]:
#Create the sustainability table in the database

name_Table = "Sus_df"

# Create table statement
sqlCreateTable = "create table if not exists "+name_Table+" (country VARCHAR(50), date INT, Forest_area FLOAT8, Cereal_yield FLOAT8, Current_account_balance FLOAT8, Foreign_direct_investment FLOAT8, Net_assistance_and_aid FLOAT8, Access_clean_fuels FLOAT8, Access_electricity FLOAT8, Renewable_electricity_output FLOAT8, CO2_emissions FLOAT8, air_pollution FLOAT8, Mammal_species_threatened FLOAT8, Population_in_slums FLOAT8, water_stress FLOAT8, Renewable_freshwater_resources FLOAT8, Terrestrial_protected_areas FLOAT8, Commercial_bank_branches FLOAT8, Broad_money_growth FLOAT8, Inflation FLOAT8, Account_ownership_financial_institution FLOAT8, Account_ownership_financial_institution_female FLOAT8, Research_development_expenditure FLOAT8, New_business_density FLOAT8, Investment_in_energy FLOAT8, Investment_in_clean_water FLOAT8, Individuals_using_Internet FLOAT8, Exports_of_goods_annual_growth FLOAT8, Exports_of_goods FLOAT8, Imports_of_goods_annual_growth FLOAT8, GDP FLOAT8, GDP_growth FLOAT8, GDP_per_capita FLOAT8, GDP_per_capita_growth FLOAT8, Literacy_rate_youth_female FLOAT8, Literacy_rate_adult_total FLOAT8, School_enrollment_preprimary FLOAT8, School_enrollment_primary_and_secondary_GPI FLOAT8, Lower_secondary_completion_rate FLOAT8, Educational_attainment_post_secondary_female FLOAT8, Educational_attainment_Bachelor FLOAT8, Adolescents_out_of_school FLOAT8, Total_alcohol_consumption_per_capita FLOAT8, Prevalence_anemia_among_women FLOAT8, Prevalence_HIV FLOAT8, Mortality_rate_under_5 FLOAT8, Mortality_CVD_cancer_diabetes FLOAT8, Mortality_rate_neonatal FLOAT8, People_using_basic_drinking_water FLOAT8, People_using_safely_managed_drinking_water FLOAT8, Female_genital_mutilation_prevalence FLOAT8, Prevalence_underweight_children_under_five FLOAT8, Suicide_mortality_rate FLOAT8, High_health_care_expenditure FLOAT8, Unemployment FLOAT8, Moderate_severe_food_insecurity FLOAT8, Women_married_by_age_18 FLOAT8, Urban_population_growth FLOAT8, Battle_related_deaths FLOAT8, Intentional_homicides_per_pop FLOAT8);"

# Create the table if it doesn't already exists
cursor.execute(sqlCreateTable)

In [45]:
#Create the political instability table in the database

name_Table = "Instability_df"

# Create table statement
sqlCreateTable = "create table if not exists "+name_Table+" (id CHAR(3), country VARCHAR(50), date INT, Political_Stability FLOAT8, Political_Stability_SE FLOAT8, lat FLOAT8, lon FLOAT8);"

# Create the table if it doesn't already exists
cursor.execute(sqlCreateTable)

In [48]:
#Function to add the rows of the dataframe into the postgreSQL table
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    # Change the values to reflect the correct number of columns in the dataframe. Uncomment the correct query before running.
    #1st query is for the political instability  table
    #query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s, %%s)" % (table, cols)
    #2nd query is for the sustainability table
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [47]:
#call and run the function execute many specifying the connection, dataframe and table to add the data to. Change the values section of the function to represent the correct number of columns.   
execute_many(conn, PT_new_df, 'public.Instability_df')

execute_many() done


In [49]:
#call and run the function execute many specifying the connection, dataframe and table to add the data to. Change the values section of the function to represent the correct number of columns.   
execute_many(conn, sus_features_df, 'public.Sus_df')

execute_many() done
